# 1. Imports

In [ ]:
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_path = 'gdrive/My Drive/Hackathon_2023/' # To be adapted by the user

Mounted at /content/gdrive


In [ ]:
import datetime
import gensim.downloader
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import random
import tensorflow

from lxml import etree
!pip install prettyprinter
from nltk.tokenize import TreebankWordTokenizer
from nltk import tokenize
from numpy.random import seed as numpy_seed
from prettyprinter import pprint
from sklearn.metrics import classification_report
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, TimeDistributed, Dense
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 395.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 3.8 MB/s eta 0:00:00


# 2. Dataset

In [ ]:
train_data = pd.read_csv(root_path + "train.csv")

In [ ]:
X_train = train_data.text
y_train = train_data.label

In [ ]:
X_train_word_tokenized = X_train.apply(word_tokenize)
X_train_sent_tokenized = X_train.apply(sent_tokenize)

In [ ]:
X_train_word_tokenized.iloc[0]

['This',
 'movie',
 'is',
 'a',
 'journey',
 'through',
 'the',
 'mind',
 'of',
 'a',
 'screenwriter',
 'caught',
 'in',
 'his',
 'own',
 'paradoxical',
 'philosophy',
 '.',
 'He',
 'examines',
 'the',
 'ever',
 'illusive',
 'question',
 'of',
 "'who",
 'am',
 'I',
 "'",
 'and',
 "'what",
 'is',
 'I',
 '?',
 "'",
 'It',
 "'s",
 'a',
 'courageous',
 'and',
 'thought',
 'provoking',
 'enterprise',
 '.',
 'There',
 'is',
 'a',
 'shipload',
 'of',
 'beautiful',
 'images',
 ',',
 'dream-inspired',
 ',',
 'Escher-like',
 'paradoxes',
 'reminiscent',
 'of',
 'the',
 'hand',
 'drawing',
 'itself',
 ',',
 'or',
 'rather',
 ',',
 'erasing',
 'itself',
 '.',
 'More',
 'and',
 'more',
 'we',
 'follow',
 'the',
 'writer',
 'in',
 'his',
 'agony',
 'over',
 'what',
 'to',
 'say',
 'and',
 'what',
 'to',
 'film',
 ',',
 'we',
 'see',
 'him',
 'phoning',
 'with',
 'his',
 'wife',
 'who',
 'left',
 'for',
 'Peru',
 ',',
 'leaving',
 'him',
 'to',
 'take',
 'care',
 'of',
 'their',
 'baby',
 ',',
 'a',


In [ ]:
X_train_sent_tokenized.head(3)

0    [This movie is a journey through the mind of a...
1    ['Water' (2005), the final part of Toronto-bas...
2    [This, which was shown dubbed in Italian at a ...
Name: text, dtype: object

In [ ]:
import gensim

In [ ]:
models_to_test = set(list(gensim.downloader.info()['models'].keys()))

In [ ]:
def out_of_vocabulary(embeddings_model, X_train_word_tokenized):
  oovs = []
  tokens = []
  for review in X_train_word_tokenized:
    for token in review:
      if token not in tokens:
        tokens.append(token)
      try:
        embedding = embeddings_model[token]
      except KeyError:
        if token not in oovs:
          oovs.append(token)

    count_token = len(tokens)
    count_oov = len(oovs)

    result = count_oov/count_token
    return result

In [ ]:
models_to_test

{'__testing_word2vec-matrix-synopsis',
 'conceptnet-numberbatch-17-06-300',
 'fasttext-wiki-news-subwords-300',
 'glove-twitter-100',
 'glove-twitter-200',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-wiki-gigaword-50',
 'word2vec-google-news-300',
 'word2vec-ruscorpora-300'}

In [ ]:
embeddings_model = gensim.downloader.load('glove-twitter-25')
score_oov = out_of_vocabulary(embeddings_model,X_train_word_tokenized)

[==================================================] 100.0% 104.8/104.8MB downloaded


In [ ]:
embeddings_model50 = gensim.downloader.load('glove-twitter-50')
score_oov50 = out_of_vocabulary(embeddings_model50,X_train_word_tokenized)

In [ ]:
score_oov50

0.15862068965517243

In [ ]:
embeddings_model['hello','hello']

array([-0.77069  ,  0.12827  ,  0.33137  ,  0.0050893, -0.47605  ,
       -0.50116  ,  1.858    ,  1.0624   , -0.56511  ,  0.13328  ,
       -0.41918  , -0.14195  , -2.8555   , -0.57131  , -0.13418  ,
       -0.44922  ,  0.48591  , -0.6479   , -0.84238  ,  0.61669  ,
       -0.19824  , -0.57967  , -0.65885  ,  0.43928  , -0.50473  ],
      dtype=float32)

# New section

In [ ]:
%%time

vectors_per_line = []

# Iterate through each line in X_train_word_tokenized
for line in X_train_word_tokenized:
    # Initialize a list to store the word vectors for this line
    line_vectors = []

    # For each word in the line, get its word vector if available
    for word in line:
        if word in embeddings_model:
            word_vector = embeddings_model[word]
            line_vectors.append(word_vector)

    # Append the list of word vectors for this line to the overall list
    vectors_per_line.append(line_vectors)

CPU times: user 17 s, sys: 672 ms, total: 17.7 s
Wall time: 17.9 s


In [ ]:
average_vectors = []

# Iterate through the list of word vectors for each line
for line_vectors in vectors_per_line:
    # Check if there are word vectors for this line
    if line_vectors:
        # Calculate the average word vector for this line
        avg_vector = np.mean(line_vectors, axis=0)
        average_vectors.append(avg_vector)
    else:
        # If there are no word vectors for the line, you can add a placeholder vector or handle it as needed
        # For example, you can add a vector of zeros as a placeholder
        avg_vector = np.zeros(embeddings_model.vector_size)
        average_vectors.append(avg_vector)

In [ ]:
len(average_vectors)

30000

In [ ]:
average_vectors[0]

array([-1.4272937e-02,  5.4409574e-03,  4.0613063e-02,  1.2869835e-01,
       -1.6128452e-01,  7.1491026e-03,  1.0654520e+00, -1.4623122e-01,
       -2.0290756e-01, -8.6695023e-02, -3.2035378e-03,  2.6582807e-01,
       -4.4781170e+00,  9.8431386e-02,  1.5041010e-01,  4.1246057e-02,
        1.4566056e-01, -4.5048822e-02, -1.7163524e-01, -3.1990290e-01,
       -9.7695217e-02,  1.5836792e-01, -1.7303239e-01, -4.6207208e-02,
       -3.0433354e-01], dtype=float32)

In [ ]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
average_vectors = np.array(average_vectors)
average_vectors[0]

array([-1.4272937e-02,  5.4409574e-03,  4.0613063e-02,  1.2869835e-01,
       -1.6128452e-01,  7.1491026e-03,  1.0654520e+00, -1.4623122e-01,
       -2.0290756e-01, -8.6695023e-02, -3.2035378e-03,  2.6582807e-01,
       -4.4781170e+00,  9.8431386e-02,  1.5041010e-01,  4.1246057e-02,
        1.4566056e-01, -4.5048822e-02, -1.7163524e-01, -3.1990290e-01,
       -9.7695217e-02,  1.5836792e-01, -1.7303239e-01, -4.6207208e-02,
       -3.0433354e-01], dtype=float32)

In [ ]:
average_vectors = np.array(average_vectors)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(average_vectors, y_train, test_size=0.2, random_state=42)

# Define the logistic regression model
logistic_regression = LogisticRegression()

# Define the hyperparameter grid to search over
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'penalty': ['l1', 'l2'],  # Regularization type
    'max_iter': [100, 1000, 10000]  # Maximum number of iterations
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=logistic_regression, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit the model with grid search to find the best parameters
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Set Accuracy:", test_accuracy)

ValueError: ignored

In [ ]:
import joblib

In [ ]:
joblib.dump(best_model, 'hugo_v1.pkl')

['hugo_v1.pkl']